In [1]:
import pandas as pd
import numpy as np
import os
import csv
import ast
from collections import Counter
import geopandas as gpd
import h5py
from datetime import datetime, timedelta
#### personal packages####
import sys

In [2]:
cluster_name = {0: 'Glocery&Pharmacies', 1: 'Retails', 2: 'Arts&Entertainment', 3: 'Restaurants&Bars',
                4: 'Educations', 5: 'Healthcares', 6: 'others'}

In [3]:
df_flow = pd.read_csv('../results/outflow_7c_wk_raw_loss_visitors.csv')
df_flow['MODZCTA'] = df_flow['MODZCTA'].astype(str)

In [4]:
cluster_cols = [cluster_name[0]+'_{}'.format(i) for i in range(56)] 
# 56 weeks from 2020-02-04 to 2021-03-02
# only need 53 weeks from 2020-02-24 to 2021-02-02
df_vist_cluster_i = df_flow[['MODZCTA','mod_idx']+cluster_cols].copy()

final_df = pd.melt(df_vist_cluster_i, id_vars=['MODZCTA','mod_idx'], value_vars = cluster_cols[3:3+51], var_name='week', value_name=cluster_name[0]+'_visits_weekly')
#### 
# Correctly adjust the code to perform the intended replacement
final_df['week'] = final_df['week'].apply(lambda x: (int(x.split('_')[1]) - 3))

for c_idx in range(1,7):
    print(cluster_name[c_idx])
    cluster_cols = [cluster_name[c_idx]+'_{}'.format(i) for i in range(56)]
    df_vist_cluster_i = df_flow[['MODZCTA', 'mod_idx']+cluster_cols].copy()
    temp_df = pd.melt(df_vist_cluster_i, id_vars=['MODZCTA', 'mod_idx'], value_vars = cluster_cols[3:3+51], 
                    var_name='week', value_name=cluster_name[c_idx]+'_visits_weekly')
    temp_df['week'] = temp_df['week'].apply(lambda x: (int(x.split('_')[1]) - 3))
    final_df = pd.merge(final_df, temp_df, on=['MODZCTA','mod_idx','week'])

Retails
Arts&Entertainment
Restaurants&Bars
Educations
Healthcares
others


Merge all the other data at the zipcode level

In [5]:
### 2 read scores data at zipcode level#########
final_df['MODZCTA'] = final_df['MODZCTA'].astype(int)
df_mod_score = pd.read_csv('../results/modzcta_scores.csv')
df_2 = final_df.merge(df_mod_score, right_on = 'MODZCTA', left_on='MODZCTA',how='left')
df_2.dropna(subset=['score_median'],inplace=True)
print(len(df_2))

6477


In [7]:
##### 3 social economic data ######
df_sed_nyc = pd.read_csv('../Data/sed_nyc_modzcta_2021.csv')
# df_sed_nyc = pd.read_csv('../Data/sed_old.csv')
df_3 = df_2.merge(df_sed_nyc, right_on = 'MODZCTA', left_on='MODZCTA',how='left')
print(len(df_3))

6477


In [8]:
### 4 merge with borough cases ###
covid_cases = pd.read_csv('../data/cases_borough_weekly.csv')
for col in ['CASE_COUNT','DEATH_COUNT', 'borough_case_count', 'borough_death_count']:
    covid_cases[col] = covid_cases[col].astype(float)
    covid_cases.loc[covid_cases[col]==0,col] = 0.1
    covid_cases[col+'_log'] = np.log(covid_cases[col])

df_4 = df_3.merge(covid_cases, on=['borough', 'week'], how='left')
print(len(df_4))

KeyError: 'borough'

In [12]:
### 5 merge with policies ####
df_policies = pd.read_csv('../data/stringency_index_ny_wk.csv')
visits_scores_wk = df_4.merge(df_policies, on='week', how='left')
print(len(visits_scores_wk))

6477


In [14]:
visits_scores_wk.to_csv('../data/unpivot_merged_data_raw_loss_visitor.csv',index=False)

In [15]:
visits_scores_wk.columns

Index(['MODZCTA', 'mod_idx', 'week', 'Glocery&Pharmacies_visits_weekly',
       'Retails_visits_weekly', 'Arts&Entertainment_visits_weekly',
       'Restaurants&Bars_visits_weekly', 'Educations_visits_weekly',
       'Healthcares_visits_weekly', 'others_visits_weekly',
       'NEIGHBORHOOD_NAME', 'COVID_CASE_COUNT', 'COVID_CASE_RATE',
       'POP_DENOMINATOR', 'COVID_DEATH_COUNT', 'COVID_DEATH_RATE',
       'PERCENT_POSITIVE', 'TOTAL_COVID_TESTS', 'borough',
       'COVID_CASE_COUNT_log', 'COVID_DEATH_COUNT_log', 'score_mean',
       'score_median', 'score_std', 'gain_bias_mean', 'gain_bias_median',
       'gain_bias_std', 'loss_bias_mean', 'loss_bias_median', 'loss_bias_std',
       'loss_aversion_scores_mean', 'loss_aversion_scores_median',
       'loss_aversion_scores_std', 'rescale_avail_mean',
       'rescale_avail_median', 'rescale_avail_std', 'rescale_realistic_mean',
       'rescale_realistic_median', 'rescale_realistic_std', 'agency_mean',
       'agency_median', 'agency_std',

In [16]:
visits_scores_wk.shape

(6477, 71)

In [13]:
df_zip = pd.read_csv('../results/modzcta_zip_counts.csv')
zip_selected = df_zip[df_zip['Count']>10]['MODZCTA'].values
visits_scores_wk[visits_scores_wk['MODZCTA'].isin(zip_selected)].to_csv('../data/unpivot_merged_data_raw_loss_visitor_s.csv',index=False)